In [2]:
from IPython.display import Image, display

import os
import autogen
from autogen.coding import LocalCommandLineCodeExecutor
from autogen_ext.models.openai import OpenAIChatCompletionClient

from dotenv import load_dotenv

load_dotenv()

config_list = [
    {
        'model': 'gpt-4o',
        'api_key':  os.getenv("OPENAI_API_KEY"),
        'tags': ['tool', 'gpt-4'],
    },
    {
        'model': 'gemini-1.5-pro',
        'api_key': os.getenv("GEMINI_API_KEY"),
        'api_type': 'google',
        'tags': ['tool', 'gemini'],
    },
    {
        'model': 'gemini-1.5-flash',
        'api_key': os.getenv("GEMINI_API_KEY"),
        'api_type': 'google',
        'tags': ['tool', 'gemini'],
    },
    {
        'model': 'gemini-1.0-pro',
        'api_key': os.getenv("GEMINI_API_KEY"),
        'api_type': 'google',
        'tags': ['gemini'],
    }
]

import json
config_list_file_name = ".config_list"
with open(config_list_file_name, "w") as file:
    json.dump(config_list, file, indent=4)

gpt_config_list = autogen.config_list_from_json(config_list_file_name, filter_dict={"tags": ["gpt-4"]})
gpt_llm_config = {"config_list": gpt_config_list, "timeout": 120}

gemini_config_list = autogen.config_list_from_json(config_list_file_name, filter_dict={"tags": ["gemini"]})
gemini_llm_config = {"config_list": gemini_config_list, "timeout": 120}

In [10]:
import os
import json
import autogen
from typing import List, Dict, Any, Optional
from utils import search_arxiv, create_pdf

# Create a UserProxyAgent with a better configuration for function execution
user_proxy = autogen.UserProxyAgent(
    name="User",
    system_message="A human user who needs a technical review document.",
    code_execution_config={
        "last_n_messages": 3, 
        "work_dir": "paper_review",
        "use_docker": False  # Set to True if you want to use Docker for isolation
    },
    human_input_mode="NEVER"
)

# Researcher agent with improved system message
researcher = autogen.AssistantAgent(
    name="Researcher",
    system_message="""You are a researcher specialized in finding relevant research papers.
    You're expert at retrieving and analyzing scientific publications.
    When asked to search for papers, ALWAYS import and use the search_arxiv function from utils.py
    by writing a Python script that imports and calls this function with appropriate parameters.
    Format your findings as a structured report with clear sections and paper summaries.
    """,
    llm_config={"config_list": gemini_config_list, "timeout": 120}
)

# Writer agent
writer = autogen.AssistantAgent(
    name="Writer",
    system_message="""You are a technical writer specialized in creating comprehensive review documents.
    You take research papers and organize them into a structured review document.
    Your reviews include:
    1. Executive summary
    2. Introduction to the field
    3. Analysis of current research directions
    4. Detailed summaries of key papers
    5. Comparison of methodologies
    6. Future research directions
    7. Conclusion
    8. References (in IEEE format)
    Your writing is clear, precise, and technical but accessible.""",
    llm_config={"config_list": gemini_config_list, "timeout": 120}
)

# PDF creator agent
pdf_creator = autogen.AssistantAgent(
    name="PDFCreator",
    system_message="""You are an expert in creating PDF documents using Python.
    You take the final review document content and format it into a professional PDF.
    ALWAYS import and use the create_pdf function from utils.py by writing a script
    that imports and calls this function with appropriate content structure.""",
    llm_config={"config_list": gemini_config_list, "timeout": 120}
)

# Create a function to run the entire workflow
def run_review_workflow(topic: str) -> str:
    """
    Execute the complete workflow to generate a technical review document.
    
    Args:
        topic: The research topic to generate a review for
        
    Returns:
        Path to the generated PDF
    """
    # Create the working directory
    # os.makedirs("paper_review", exist_ok=True)
    
    # Copy utils.py to the working directory to ensure it's accessible
    # import shutil
    # shutil.copy("utils.py", os.path.join("paper_review", "utils.py"))
    
    # Create a group chat
    groupchat = autogen.GroupChat(
        agents=[user_proxy, researcher, writer, pdf_creator],
        messages=[],
        max_round=20  # Increase max rounds to allow for more interaction
    )
    
    # Create a group chat manager WITH an LLM config (this was missing before)
    manager = autogen.GroupChatManager(
        groupchat=groupchat,
        llm_config={"config_list": gemini_config_list}
    )
    
    search_f = r"""def search_arxiv(topic: str, max_results: int = 15) -> List[Dict[Any, Any]]:
                client = arxiv.Client()
                search = arxiv.Search(
                    query=topic,
                    max_results=max_results,
                    sort_by=arxiv.SortCriterion.SubmittedDate,
                    sort_order=arxiv.SortOrder.Descending
                )
                
                results = []
                for paper in client.results(search):
                    paper_dict = {
                        "title": paper.title,
                        "authors": [author.name for author in paper.authors],
                        "summary": paper.summary,
                        "published": paper.published.strftime("%Y-%m-%d"),
                        "url": paper.pdf_url,
                        "arxiv_id": paper.get_short_id(),
                        "categories": paper.categories
                    }
                    results.append(paper_dict)
                return results"""
    
    pdf_f = r"""def create_pdf(content: Dict[str, str], filename: str = "technical_review.pdf") -> str:
            pdf = fpdf.FPDF()
            
            # Title page
            pdf.add_page()
            pdf.set_font("Arial", "B", 24)
            pdf.cell(0, 20, content["title"], ln=True, align="C")
            pdf.set_font("Arial", "", 12)
            pdf.cell(0, 10, f"Created: {datetime.now().strftime('%Y-%m-%d')}", ln=True, align="C")
            
            # Table of contents
            pdf.add_page()
            pdf.set_font("Arial", "B", 16)
            pdf.cell(0, 10, "Table of Contents", ln=True)
            pdf.set_font("Arial", "", 12)
            
            y_position = pdf.get_y()
            for section in ["executive_summary", "introduction", "current_research", "key_papers", 
                        "methodology_comparison", "future_directions", "conclusion", "references"]:
                if section in content:
                    section_title = section.replace("_", " ").title()
                    pdf.cell(0, 8, f"{section_title}", ln=True)
            
            # Add content sections
            sections = [
                ("Executive Summary", content.get("executive_summary", "")),
                ("Introduction", content.get("introduction", "")),
                ("Current Research Directions", content.get("current_research", "")),
                ("Key Papers", content.get("key_papers", "")),
                ("Methodology Comparison", content.get("methodology_comparison", "")),
                ("Future Research Directions", content.get("future_directions", "")),
                ("Conclusion", content.get("conclusion", "")),
                ("References", content.get("references", ""))
            ]
            
            for title, text in sections:
                if text:
                    pdf.add_page()
                    pdf.set_font("Arial", "B", 16)
                    pdf.cell(0, 10, title, ln=True)
                    pdf.set_font("Arial", "", 12)
                    
                    # Split text into paragraphs and add to PDF
                    paragraphs = text.split("\n\n")
                    for para in paragraphs:
                        pdf.multi_cell(0, 8, para)
                        pdf.ln(4)
            
            # Save the PDF
            import os
            filepath = os.path.abspath(filename)
            pdf.output(filename)
            print(f"PDF created at: {filepath}")
            return filepath"""

    # Start the chat with clear instructions
    user_proxy.initiate_chat(
        manager,
        message=f"""
        I need a comprehensive technical review document on "{topic}".
        
        Follow these steps exactly:
        
        1. Researcher: Search for the most relevant papers on this topic by writing code to import and use the search_arxiv function from utils.py.
           For example:
           ```python
           {search_f}
           ```
        2. Researcher: Analyze the results and identify the 5-7 most important papers returned by the search_arxiv function.
        
        3. Writer: Create a comprehensive technical review based on these papers.
        
        4. PDFCreator: Try to read the code from utils.py file first. There are two functions search_arxiv and create_pdf.
           PDFCreator: Create a PDF by writing code to import and use the create_pdf function from utils.py:
           ```python
           {pdf_f}
           ```
           
           by providing the actual values of the keys retrieved from the Writer. 
           Filename should be "{topic.replace(' ', '_')}_review.pdf"
           
        5. The final output should be a PDF file saved in the working directory.
        """
    )
    
    # Return the expected filename
    return f"{topic.replace(' ', '_')}_review.pdf"

# Usage example
if __name__ == "__main__":
    # Set your OpenAI API key as an environment variable before running
    # os.environ["OPENAI_API_KEY"] = "your-api-key"
    
    topic = "quantum machine learning"
    pdf_path = run_review_workflow(topic)
    print(f"Workflow completed. Review document should be at: {pdf_path}")

User (to chat_manager):


        I need a comprehensive technical review document on "quantum machine learning".
        
        Follow these steps exactly:
        
        1. Researcher: Search for the most relevant papers on this topic by writing code to import and use the search_arxiv function from utils.py.
           For example:
           ```python
           def search_arxiv(topic: str, max_results: int = 15) -> List[Dict[Any, Any]]:
                client = arxiv.Client()
                search = arxiv.Search(
                    query=topic,
                    max_results=max_results,
                    sort_by=arxiv.SortCriterion.SubmittedDate,
                    sort_order=arxiv.SortOrder.Descending
                )
                
                results = []
                for paper in client.results(search):
                    paper_dict = {
                        "title": paper.title,
                        "authors": [author.name for author in paper.authors


Next speaker: Researcher

Researcher (to chat_manager):

```python
import arxiv
from typing import List, Dict, Any
from datetime import datetime
import fpdf
import os

def search_arxiv(topic: str, max_results: int = 15) -> List[Dict[Any, Any]]:
    client = arxiv.Client()
    search = arxiv.Search(
        query=topic,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate,
        sort_order=arxiv.SortOrder.Descending
    )

    results = []
    for paper in client.results(search):
        paper_dict = {
            "title": paper.title,
            "authors": [author.name for author in paper.authors],
            "summary": paper.summary,
            "published": paper.published.strftime("%Y-%m-%d"),
            "url": paper.pdf_url,
            "arxiv_id": paper.get_short_id(),
            "categories": paper.categories
        }
        results.append(paper_dict)
    return results

def create_pdf(content: Dict[str, str], filename: str = "technical_revi